In [1]:
import requests
response = requests.get("https://www.cars24.com/")

In [23]:
url = "https://www.cars24.com/buy-used-cars-bangalore/"

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Initialize the WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get(url)  # Replace with the actual URL

try:
    # Step 1: Explicit wait for a key element to ensure initial content is loaded
    # Replace '.data-class' with the actual CSS selector of an element that indicates data is loaded
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.data-class'))
    )

    # Step 2: Handle infinite scroll to load all content
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Brief pause to allow new content to load
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:  # No more content to load
            break
        last_height = new_height
        
    html = driver.page_source

except Exception as e:
    print(f"An error occurred: {e}")
    html = driver.page_source  # Capture whatever is available on error

finally:
    driver.quit()

# Optional: Save the HTML to a file for inspectio

An error occurred: Message: 
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x634103
	0x634144
	0x43e71d
	0x48a03d
	0x48a41b
	0x4d17f2
	0x4ac954
	0x4cee17
	0x4ac706
	0x47da30
	0x47ed54
	0x8a57b4
	0x8a098a
	0x65c392
	0x64c4c8
	0x65324d
	0x63c478
	0x63c63c
	0x6267ca
	0x75fd5d49
	0x771dd6db
	0x771dd661



In [17]:
with open("cars.html", "w") as f:
    f.write(html)

In [21]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time

# Configure Chrome options for better performance
chrome_options = Options()
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.add_argument('--start-maximized')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)

# Initialize driver
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=chrome_options
)

# Set page load timeout
driver.set_page_load_timeout(30)

try:
    # Replace 'banglore' with your actual URL
    url = url  # Make sure this is a valid URL string
    driver.get(url)
    
    # Wait for page to load completely
    time.sleep(5)
    
    # Wait for main scroll container with multiple possible selectors
    scroll_container = None
    selectors = [
        "div.qk-container",
        "div[class*='scroll']",
        "div[class*='container']",
        "body"  # Fallback to body if specific container not found
    ]
    
    for selector in selectors:
        try:
            scroll_container = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, selector))
            )
            print(f"Found scroll container with selector: {selector}")
            break
        except TimeoutException:
            continue
    
    if not scroll_container:
        print("No scroll container found, using page body")
        scroll_container = driver.find_element(By.TAG_NAME, "body")
    
    # Enhanced scrolling with multiple strategies
    print("Starting scroll process...")
    last_height = 0
    scroll_attempts = 0
    max_scroll_attempts = 50  # Prevent infinite loops
    no_change_count = 0
    max_no_change = 3  # Stop after 3 consecutive no-change scrolls
    
    while scroll_attempts < max_scroll_attempts:
        # Get current height before scroll
        current_height = driver.execute_script(
            "return arguments[0].scrollHeight;", 
            scroll_container
        )
        
        # Scroll down
        driver.execute_script(
            "arguments[0].scrollTo(0, arguments[0].scrollHeight);",
            scroll_container
        )
        
        # Wait for content to load
        time.sleep(3)  # Increased wait time
        
        # Try clicking "Load More" or "Show More" buttons if they exist
        try:
            load_more_buttons = driver.find_elements(
                By.XPATH, 
                "//*[contains(text(), 'Load More') or contains(text(), 'Show More') or contains(text(), 'View More')]"
            )
            for button in load_more_buttons:
                if button.is_displayed():
                    driver.execute_script("arguments[0].click();", button)
                    print("Clicked 'Load More' button")
                    time.sleep(2)
        except Exception as e:
            pass
        
        # Get new height after scroll
        new_height = driver.execute_script(
            "return arguments[0].scrollHeight;", 
            scroll_container
        )
        
        print(f"Scroll {scroll_attempts + 1}: Height {current_height} -> {new_height}")
        
        # Check if height changed
        if new_height == last_height:
            no_change_count += 1
            print(f"No change in height (count: {no_change_count})")
            
            # Try scrolling up slightly and back down to trigger lazy loading
            driver.execute_script(
                "arguments[0].scrollTo(0, arguments[0].scrollHeight - 500);",
                scroll_container
            )
            time.sleep(1)
            driver.execute_script(
                "arguments[0].scrollTo(0, arguments[0].scrollHeight);",
                scroll_container
            )
            time.sleep(2)
            
            if no_change_count >= max_no_change:
                print("Reached bottom of page")
                break
        else:
            no_change_count = 0  # Reset counter if height changed
        
        last_height = new_height
        scroll_attempts += 1
    
    # Final wait to ensure all content is loaded
    time.sleep(5)
    
    # Get the page source
    html = driver.page_source
    
    print(f"Successfully scraped page. HTML length: {len(html)} characters")
    
    # Optional: Save HTML to file for inspection
    with open("scraped_page.html", "w", encoding="utf-8") as f:
        f.write(html)
    print("HTML saved to scraped_page.html")
    
    # You can now parse the HTML with BeautifulSoup or extract specific elements
    # Example:
    # from bs4 import BeautifulSoup
    # soup = BeautifulSoup(html, 'html.parser')
    # data = soup.find_all('div', class_='your-data-class')
    
except TimeoutException as e:
    print(f"Timeout error: {e}")
except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()
finally:
    driver.quit()
    print("Driver closed")

Found scroll container with selector: div[class*='container']
Starting scroll process...
Scroll 1: Height 179 -> 179
Scroll 2: Height 179 -> 179
No change in height (count: 1)
Scroll 3: Height 179 -> 179
No change in height (count: 2)
Scroll 4: Height 179 -> 179
No change in height (count: 3)
Reached bottom of page
Successfully scraped page. HTML length: 3333106 characters
HTML saved to scraped_page.html
Driver closed


In [25]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import json

# Configure Chrome options with performance logging
chrome_options = Options()
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.add_argument('--start-maximized')
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)

# Enable performance logging to capture network requests
chrome_options.set_capability('goog:loggingPrefs', {'performance': 'ALL'})

# Initialize driver
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=chrome_options
)

try:
    # Replace with your actual URL
    url = url
    print(f"Loading URL: {url}")
    driver.get(url)
    
    # Wait for network idle (no new requests for 2 seconds)
    print("Waiting for network to settle...")
    time.sleep(5)
    
    # Wait for document ready state
    WebDriverWait(driver, 30).until(
        lambda d: d.execute_script('return document.readyState') == 'complete'
    )
    print("Document ready state: complete")
    
    # Wait for specific elements that indicate data is loaded
    # Adjust these selectors based on your target website
    possible_selectors = [
        "div[class*='product']",
        "div[class*='item']",
        "div[class*='card']",
        "li[class*='list']",
        "article",
        "div[data-testid]",
        "div[id*='item']"
    ]
    
    element_found = False
    for selector in possible_selectors:
        try:
            elements = WebDriverWait(driver, 5).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, selector))
            )
            if elements:
                print(f"✓ Found {len(elements)} elements with selector: {selector}")
                element_found = True
                break
        except TimeoutException:
            continue
    
    if not element_found:
        print("⚠ No standard content elements found, trying alternate approach...")
    
    # Scroll to load all content
    print("\nStarting intelligent scroll...")
    last_count = 0
    scroll_pause = 3
    max_scrolls = 30
    
    for i in range(max_scrolls):
        # Scroll down
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause)
        
        # Count visible elements
        try:
            current_count = len(driver.find_elements(By.CSS_SELECTOR, "div, article, li"))
            print(f"Scroll {i+1}: Found {current_count} total elements")
            
            if current_count == last_count:
                print("No new elements loaded, stopping scroll")
                break
            last_count = current_count
        except:
            pass
        
        # Try to click any "Load More" buttons
        try:
            load_more = driver.find_elements(By.XPATH, 
                "//*[contains(translate(text(), 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), 'load more') or " +
                "contains(translate(text(), 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), 'show more')]"
            )
            for btn in load_more:
                if btn.is_displayed():
                    driver.execute_script("arguments[0].click();", btn)
                    print("Clicked 'Load More' button")
                    time.sleep(3)
        except:
            pass
    
    # Final wait
    time.sleep(5)
    
    # Get the rendered HTML
    html = driver.page_source
    print(f"\n✓ Captured HTML: {len(html)} characters")
    
    # Save HTML
    with open("scraped_page_full.html", "w", encoding="utf-8") as f:
        f.write(html)
    print("✓ HTML saved to scraped_page_full.html")
    
    # Take screenshot
    driver.save_screenshot("page_final.png")
    print("✓ Screenshot saved to page_final.png")
    
    # Analyze content
    print("\n--- Content Analysis ---")
    print(f"Total HTML length: {len(html):,} characters")
    print(f"Contains <div> tags: {html.count('<div')}")
    print(f"Contains <article> tags: {html.count('<article')}")
    print(f"Contains <li> tags: {html.count('<li')}")
    print(f"Contains <script> tags: {html.count('<script')}")
    
    # Try to extract data using common patterns
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')
    
    # Remove script and style tags for cleaner content
    for script in soup(["script", "style"]):
        script.decompose()
    
    # Get text content
    text = soup.get_text(separator=' ', strip=True)
    print(f"Extracted text length: {len(text):,} characters")
    print(f"\nFirst 500 characters of content:\n{text[:500]}")
    
    # Save text content
    with open("scraped_text.txt", "w", encoding="utf-8") as f:
        f.write(text)
    print("\n✓ Text content saved to scraped_text.txt")
    
    # Optional: Capture network requests (API calls that might contain data)
    print("\n--- Network Analysis ---")
    logs = driver.get_log('performance')
    api_calls = []
    
    for log in logs:
        try:
            message = json.loads(log['message'])['message']
            if message['method'] == 'Network.responseReceived':
                url = message['params']['response']['url']
                if 'api' in url.lower() or 'json' in url.lower():
                    api_calls.append(url)
        except:
            continue
    
    if api_calls:
        print(f"Found {len(api_calls)} potential API calls:")
        for api_url in set(api_calls)[:10]:  # Show first 10 unique
            print(f"  - {api_url}")
        print("\n💡 Tip: You might be able to scrape data directly from these API endpoints!")
    else:
        print("No obvious API calls found")
    
except Exception as e:
    print(f"\n❌ Error: {e}")
    import traceback
    traceback.print_exc()
finally:
    driver.quit()
    print("\n✓ Driver closed")

Loading URL: https://www.cars24.com/buy-used-cars-bangalore/
Waiting for network to settle...
Document ready state: complete
✓ Found 1 elements with selector: div[class*='item']

Starting intelligent scroll...
Scroll 1: Found 1552 total elements
Scroll 2: Found 1552 total elements
No new elements loaded, stopping scroll

✓ Captured HTML: 1635784 characters
✓ HTML saved to scraped_page_full.html
✓ Screenshot saved to page_final.png

--- Content Analysis ---
Total HTML length: 1,635,784 characters
Contains <div> tags: 1465
Contains <article> tags: 0
Contains <li> tags: 155
Contains <script> tags: 93
Extracted text length: 31,617 characters

First 500 characters of content:
Used Cars in Bangalore - 2148 Second Hand Cars | CARS24 Bangalore Buy used car Buy Used Cars in Bangalore Change city Browse by Model Browse by Make Browse by Price Browse by Body Type Browse by Fuel Type Browse by Transmission Browse by City Browse by Model Used Hyundai i10 Cars in Bangalore Used Hyundai i20 Cars in B

Traceback (most recent call last):
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_6600\1953122559.py", line 168, in <module>
    for api_url in set(api_calls)[:10]:  # Show first 10 unique
                   ~~~~~~~~~~~~~~^^^^^
TypeError: 'set' object is not subscriptable



✓ Driver closed
